# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os

# Importing Data

In [2]:
path = r'/Users/docopeland/04 Instacart Basket Analysis'
orders = pd.read_csv(os.path.join(path, '02 Data', 'original data', 'orders.csv'))

# Data Observation

In [3]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [4]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB


In [5]:
orders.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


In [6]:
orders['eval_set'].value_counts(dropna = False)

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

# Data Wrangling

In [7]:
#dropping column 'eval_set'
orders = orders.drop(columns = ['eval_set'])

In [8]:
#renaming columns 'order_dow' and 'order_number'
orders.rename(columns = {'order_dow' : 'order_day_of_week'}, inplace = True)
orders.rename(columns = {'order_number' : 'total_customer_orders'}, inplace = True)

In [9]:
#changing order_id and user_id from a integer to a string
orders['order_id'] = orders['order_id'].astype('str')
orders['user_id'] = orders['user_id'].astype('str')

In [10]:
#viewing the data after the wrangling
orders.head()

,order_id,user_id,total_customer_orders,order_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


# Consistency Check

In [11]:
# checking for mixed data types
for col in orders.columns.tolist():
  weird = (orders[[col]].applymap(type) != orders[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (orders[weird]) > 0:
    print (col)

In [12]:
#checking for null values, there's lots of days_since_prior_order
orders.isnull().sum()

order_id                       0
user_id                        0
total_customer_orders          0
order_day_of_week              0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

In [13]:
orders['days_since_prior_order'].value_counts(dropna = False)

30.0    369323
7.0     320608
6.0     240013
4.0     221696
3.0     217005
5.0     214503
NaN     206209
2.0     193206
8.0     181717
1.0     145247
9.0     118188
14.0    100230
10.0     95186
13.0     83214
11.0     80970
12.0     76146
0.0      67755
15.0     66579
16.0     46941
21.0     45470
17.0     39245
20.0     38527
18.0     35881
19.0     34384
22.0     32012
28.0     26777
23.0     23885
27.0     22013
24.0     20712
25.0     19234
29.0     19191
26.0     19016
Name: days_since_prior_order, dtype: int64

In [14]:
#the number of unique 'user_id's matches the number of null values
#so days_since_prior_order seems to be null the first time a user makes an order
#no need to take care of this
orders['user_id'].nunique()

206209

In [15]:
#checking for duplicates, there are no duplicates
orders.duplicated().sum()

0

# Checking Data Again

In [16]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 6 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                object 
 1   user_id                 object 
 2   total_customer_orders   int64  
 3   order_day_of_week       int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 156.6+ MB


In [17]:
orders.describe()

,total_customer_orders,order_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


# Export Data

In [18]:
orders.to_csv(os.path.join(path, '02 Data', 'prepared data', 'orders_clean.csv'))